# Lab 7: Topic Modeling

---
## 1. Notebook Overview

### 1.1 Objective
Build and interpret topic models on the tweet dataset using Gensim's LDA implementation, then visualize and evaluate the discovered topics.

### 1.2 Prerequisites
This notebook assumes you have already executed:
- **Lab 2**: Data preprocessing → `../Data/multi_label/tweets_preprocessed_*.parquet`
- **Lab 3**: Language modeling
- **Lab 4**: Feature extraction → `../Data/top_1000_vocabulary.json`
- **Lab 5**: Neural network classification pipeline 
- **Lab 5 & 6**: Understanding of the dataset labels to compare topics with classes

### 1.3 Resources
- Gensim LDA documentation: https://radimrehurek.com/gensim/models/ldamodel.html
- pyLDAvis usage guide: https://pyldavis.readthedocs.io/en/latest/

### 1.4 Task Summary
- Train one or more LDA topic models with Gensim (consider preprocessing choices and reusing prior pipelines).
- Visualize the learned topics via pyLDAvis (`pyLDAvis.gensim_models.prepare(...)`).
- Interpret and critique the topics: which terms describe them? How do they align (or not) with the tweet label taxonomy?
- Experiment with multiple topic counts to see how granularity affects interpretability.

### 1.5 Section Roadmap
1. Section 2: Environment setup, data access, and preprocessing recap
2. Section 3: Build and train the Gensim LDA model
3. Section 4: Visualize topics with pyLDAvis
4. Section 5: Interpret topics, compare topic counts, and discuss insights

---
## 2. Data Loading & Preparation

### 2.1 Libraries
- `gensim` for LDA (`LdaModel`, `Dictionary`, `corpora`)
- `pyLDAvis.gensim_models` for interactive topic visualization
- `pandas`, `numpy`, `datasets` (HuggingFace) for loading the tweet splits

### 2.2 Installation Steps
1. Install/confirm `gensim`, `pyLDAvis`, and any preprocessing dependencies in the environment.
2. Ensure the preprocessed tweet file `../Data/multi_label/tweets_preprocessed_train.parquet` and HuggingFace API access (`cardiffnlp/tweet_topic_multi`) are available.

In [13]:
%pip install --quiet gensim pyLDAvis numpy pandas 

Note: you may need to restart the kernel to use updated packages.


In [14]:
import warnings
import pandas as pd
import numpy as np
import typing
import json
import re
import pyLDAvis
import pyLDAvis.gensim_models


import gensim
from pathlib import Path
from typing import Sequence, Union, List
from datasets import load_dataset
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel, CoherenceModel

warnings.filterwarnings("ignore")

TRAIN_DATA_PATH = "../Data/multi_label/tweets_preprocessed_train.parquet"
TEST_DATA_PATH = "../Data/multi_label/tweets_preprocessed_test.parquet"
VAL_DATA_PATH = "../Data/multi_label/tweets_preprocessed_validation.parquet"
VOCABULARY_PATH = "../Data/top_1000_vocabulary.json"
RANDOM_STATE = 42

In [15]:
# Load the Top 1000 vocabulary from Lab 4
print("Loading vocabulary from Lab 4...")
with open(VOCABULARY_PATH, 'r', encoding='utf-8') as f:
    vocab_data = json.load(f)

TOP_VOCABULARY = vocab_data['tokens']

print(f"✓ Loaded vocabulary from: {VOCABULARY_PATH}")
print(f"✓ Vocabulary size: {len(TOP_VOCABULARY)}")
print(f"✓ First 10 tokens: {TOP_VOCABULARY[:10]}")
print(f"✓ Description: {vocab_data. get('description', 'N/A')}")

Loading vocabulary from Lab 4...
✓ Loaded vocabulary from: ../Data/top_1000_vocabulary.json
✓ Vocabulary size: 1000
✓ First 10 tokens: ['new', 'game', 'day', 'good', 'year', 'love', 'time', 'win', 'come', 'happy']
✓ Description: Top 1000 most frequent tokens from preprocessed tweets (Lab 4)


In [16]:
# Load preprocessed training data from Lab 2
print("\nLoading preprocessed training data from Lab 2...")
df_train = pd.read_parquet(TRAIN_DATA_PATH)
print(f"Training samples: {len(df_train):,}")
print(f"Columns: {df_train.columns.tolist()}")
df_train.head(3)


Loading preprocessed training data from Lab 2...
Training samples: 5,465
Columns: ['text', 'label_name', 'label']


,text,label_name,label
0,lumber beat rapid game western division final ...,['sports'],[0 0 0 0 0 1]
1,hear eli gold announce auburn game dumbass,['sports'],[0 0 0 0 0 1]
2,phone away try look home game ticket october,['sports'],[0 0 0 0 0 1]


In [17]:
print("Loading preprocessed test and validation data...")
df_test = pd.read_parquet(TEST_DATA_PATH)
df_val = pd.read_parquet(VAL_DATA_PATH)
print(f"Test samples: {len(df_test):,}")
print(f"Validation samples: {len(df_val):,}")

Loading preprocessed test and validation data...
Test samples: 1,511
Validation samples: 178


---
## 3. Train LDA Topic Models

### 3.1 Dictionary and Corpus
Tweets are re-tokenized with the Lab 2 preprocessing steps and restricted to the Lab 4 vocabulary so LDA concentrates on the most meaningful tokens. The `Dictionary` is built directly from these filtered tokens, preserving the entire vocabulary, including rare terms for the later interpretation work.

### 3.2 Topic Search Grid
To satisfy the requirements we sweep over a `topic_grid`. When the validation split retains enough tweets after vocabulary filtering we prioritize the `CoherenceModel (c_v)` score; otherwise we fall back to validation perplexity plus manual inspection because the validation subset is relatively small.


In [18]:
def ensure_tokens(sentence: Union[Sequence[str], str]) -> List[str]:
    """Whitespace tokenizer reused from previous labs."""
    if isinstance(sentence, str):
        sentence = sentence.split()
    return list(sentence)

def tokenize_column(series, vocabulary=None):
    """Tokenize tweets and optionally restrict tokens to a curated vocabulary."""
    docs, kept_idx = [], []
    for idx, row in enumerate(series):
        tokens = ensure_tokens(row)
        if vocabulary is not None:
            tokens = [tok for tok in tokens if tok in vocabulary]
        if tokens:
            docs.append(tokens)
            kept_idx.append(idx)
    return docs, kept_idx

def normalize_label(entry):
    """Reduce multi-hot or string labels to a single representative tag."""
    if isinstance(entry, list) and entry:
        return entry[0]
    if isinstance(entry, str):
        matches = re.findall(r"[A-Za-z0-9_&]+", entry)
        if matches:
            return matches[0]
        return entry.strip()
    return "unknown"

In [19]:
# Restrict documents to the curated Lab 4 vocabulary for cleaner topics.
top_vocab = set(TOP_VOCABULARY)

train_df = pd.read_parquet(TRAIN_DATA_PATH)
val_df = pd.read_parquet(VAL_DATA_PATH)

train_tokens, train_idx = tokenize_column(train_df["text"], vocabulary=top_vocab)
val_tokens, val_idx = tokenize_column(val_df["text"], vocabulary=top_vocab)

# Align labels with the filtered token lists so later evaluation uses matching rows.
train_labels = [normalize_label(train_df["label_name"].iloc[i]) for i in train_idx]
val_labels = [normalize_label(val_df["label_name"].iloc[i]) for i in val_idx]

print(f"Docs kept → train: {len(train_tokens)}, val: {len(val_tokens)}")
print(f"Sample tokens: {train_tokens[0][:15]}")



Docs kept → train: 5458, val: 178
Sample tokens: ['beat', 'game', 'final', 'hit']


In [20]:
dictionary = gensim.corpora.Dictionary(train_tokens)

train_corpus = [dictionary.doc2bow(text) for text in train_tokens]
val_corpus = [dictionary.doc2bow(text) for text in val_tokens]

print(f"Dictionary size: {len(dictionary)}")
print(f"Train documents: {len(train_corpus)}, Validation documents: {len(val_corpus)}")


Dictionary size: 1000
Train documents: 5458, Validation documents: 178


In [21]:
def train_lda(num_topics, passes=12):
    return LdaModel(
        corpus=train_corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=RANDOM_STATE,
        chunksize=2048,
        passes=passes,
        alpha="auto",
        eta="auto",
        minimum_probability=0.0,
    )

topic_grid = [15, 19, 35, 45] # label-aligned, mid, and high-granularity k
lda_runs, rows = {}, []

for k in topic_grid:
    model = train_lda(k)
    lda_runs[k] = model

    coherence_model = CoherenceModel(
        model=model,
        texts=train_tokens, # val tokens too sparse for stable c_v
        dictionary=dictionary,
        coherence="c_v",
    )
    coherence = coherence_model.get_coherence()
    perplexity = model.log_perplexity(val_corpus)

    rows.append(
        {
            "topics": k,
            "coherence_c_v": coherence,
            "val_perplexity": perplexity,
        }
    )
    print(f"k={k:2d} → coherence={coherence:.3f}, perplexity={perplexity:.3f}")




k=15 → coherence=0.324, perplexity=-13.723
k=19 → coherence=0.367, perplexity=-14.324
k=35 → coherence=0.412, perplexity=-15.080
k=45 → coherence=0.451, perplexity=-14.982


### Topic sweep (k = 19, 35, 45) with coherence + perplexity
- 19 topics mirror the 19 supervised labels in the dataset, so we can compare them directly.  
- 35 topics act as a mid-granularity level that already splits subthemes.  
- 45 topics gave the best trade-off between perplexity and interpretable coherence in earlier runs.  
We compute coherence on `train_tokens` because the validation split shrinks too much after the vocabulary filter and would otherwise return NaN; the test set stays untouched.

In [22]:
metrics_df = (
    pd.DataFrame(rows)
    .sort_values(["coherence_c_v", "val_perplexity"], ascending=[False, True])
    .reset_index(drop=True)
)
display(metrics_df)

best_topic_count = int(metrics_df.iloc[0]["topics"])
best_model = lda_runs[best_topic_count]
print(f"Chosen topic count: {best_topic_count}")

,topics,coherence_c_v,val_perplexity
0,45,0.450866,-14.982145
1,35,0.411792,-15.080116
2,19,0.366885,-14.324221
3,15,0.323755,-13.723411


Chosen topic count: 45


---
## 4. Visualize and Inspect Topics

### 4.1 Interactive pyLDAvis
Use `pyLDAvis.gensim.prepare(...)` to render the intertopic distance map (bubble plot) and the top-term bar chart for any trained model. Bubble size encodes how prevalent a topic is in the corpus, distances highlight similarity, and the λ slider controls whether the term list emphasizes raw frequency (λ ≈ 1) or exclusivity (λ ≈ 0). Exported HTML dashboards saved in `../Data/pyldavis_k19.html` and `../Data/pyldavis_k45.html`.

### 4.2 Tabular Keyword Summary
Because notebooks are often shared as static reports, a compact table of the top-N keywords per topic is generated alongside the visualizations. It’s handy for quick scans (e.g., pasting into slides) or when pyLDAvis can’t be opened.


In [23]:
print("Preparing pyLDAvis visualization for k=19 topics ...")
panel_k19 = pyLDAvis.gensim_models.prepare(
    lda_runs[19],
    train_corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.enable_notebook()
pyLDAvis.display(panel_k19)
panel_k19

Preparing pyLDAvis visualization for k=19 topics ...


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.074984 -0.015398       1        1  5.508696
1      0.005561 -0.137226       2        1  5.237410
2     -0.075003 -0.092198       3        1  9.003852
3      0.061152  0.246046       4        1  3.468559
4      0.031191 -0.003353       5        1  6.302546
5     -0.026028 -0.029909       6        1  4.119859
6     -0.153130  0.003309       7        1  5.069482
7     -0.020812  0.240293       8        1  5.487388
8     -0.024033 -0.015003       9        1  5.041078
9     -0.043152  0.037834      10        1  4.746899
10    -0.071147 -0.210000      11        1  4.140039
11    -0.066643 -0.086833      12        1  4.750864
12    -0.052945  0.171739      13        1  4.290138
13     0.297069 -0.023258      14        1  5.847954
14     0.019794 -0.079214      15        1  4.913450
15     0.075542 -0.128769      16        1  5.951241
16     0.336864  0.008177      17        1  4.178414
17    -0.100542  0.059222      18        1  5.501116
18    -0.118756  0.054540      19        1  6.441016, topic_info=       Term        Freq       Total Category  logprob  loglift
50      new  532.000000  532.000000  Default  30.0000  30.0000
163     day  448.000000  448.000000  Default  29.0000  29.0000
419   music  374.000000  374.000000  Default  28.0000  28.0000
59    video  277.000000  277.000000  Default  27.0000  27.0000
207     red  260.000000  260.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
84   season   30.714935  145.520665  Topic19  -4.5679   1.1869
392     fan   28.602926  116.935935  Topic19  -4.6391   1.3344
83    great   31.342678  275.780887  Topic19  -4.5476   0.5679
99     hope   28.380747  163.035764  Topic19  -4.6469   0.9942
191   think   27.979890  189.794846  Topic19  -4.6611   0.8280

[904 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
481       5  0.708663    absolute
481      12  0.265749    absolute
669       3  0.097165  absolutely
669       4  0.453435  absolutely
669      18  0.226717  absolutely
...     ...       ...         ...
753      15  0.949911          yo
426       7  0.183674        york
426      14  0.734696        york
426      19  0.061225        york
212      10  0.937205        zone

[1891 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [25]:
print("Preparing pyLDAvis visualization for k=45 topics ...")
panel_k45 = pyLDAvis.gensim_models.prepare(
    lda_runs[45],
    train_corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.display(panel_k45)
panel_k45

Preparing pyLDAvis visualization for k=45 topics ...


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.110491 -0.176911       1        1  2.955433
1      0.050844 -0.049425       2        1  2.418471
2      0.145642  0.088730       3        1  3.726276
3      0.020653  0.123763       4        1  1.849328
4     -0.111743  0.024783       5        1  1.846100
5     -0.013042 -0.064173       6        1  1.908931
6      0.057177 -0.121386       7        1  1.944285
7      0.050751  0.014796       8        1  1.435349
8      0.019471 -0.022198       9        1  2.331784
9     -0.036916 -0.128947      10        1  2.098711
10     0.030192  0.185945      11        1  2.119921
11     0.027501  0.075932      12        1  2.176593
12    -0.046366 -0.066427      13        1  1.881698
13    -0.237475  0.073707      14        1  2.690320
14    -0.000061 -0.028287      15        1  1.756726
15     0.009672  0.067872      16        1  2.200368
16    -0.051497 -0.068452      17        1  2.085985
17    -0.007604  0.041936      18        1  2.397038
18     0.130913 -0.059929      19        1  2.898478
19     0.016809  0.025264      20        1  1.953962
20     0.152599 -0.004131      21        1  2.173533
21    -0.249278  0.064061      22        1  2.935177
22    -0.001529 -0.002333      23        1  2.026372
23    -0.008535 -0.101712      24        1  2.543794
24     0.117385  0.045936      25        1  3.391957
25     0.081574 -0.073231      26        1  2.372018
26    -0.133711  0.014358      27        1  1.793023
27    -0.125406 -0.008318      28        1  1.727477
28     0.017372  0.179710      29        1  2.237725
29     0.057036  0.153088      30        1  2.150902
30     0.077474 -0.001466      31        1  2.541820
31     0.056236  0.162832      32        1  1.745617
32    -0.204394  0.020892      33        1  2.661070
33     0.037266  0.005333      34        1  2.032652
34     0.138139  0.083307      35        1  2.727796
35    -0.039771 -0.087228      36        1  1.574376
36    -0.028786  0.015019      37        1  2.113189
37     0.027180 -0.086452      38        1  2.119667
38     0.053453  0.072212      39        1  2.749045
39    -0.065902 -0.055184      40        1  1.527090
40    -0.056021  0.041453      41        1  1.424812
41     0.009171 -0.105389      42        1  2.139073
42    -0.005941 -0.080015      43        1  2.038510
43    -0.064075 -0.049360      44        1  2.264552
44    -0.006946 -0.139976      45        1  2.312995, topic_info=        Term        Freq       Total Category  logprob  loglift
292     love  461.000000  461.000000  Default  30.0000  30.0000
163      day  440.000000  440.000000  Default  29.0000  29.0000
50       new  544.000000  544.000000  Default  28.0000  28.0000
42     watch  366.000000  366.000000  Default  27.0000  27.0000
419    music  369.000000  369.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
432  country   15.542767   76.268609  Topic45  -4.2249   2.1760
102     need   20.027710  218.391010  Topic45  -3.9714   1.3775
559     help   14.092530  107.485496  Topic45  -4.3228   1.7349
79      time   14.531501  354.784202  Topic45  -4.2922   0.5714
334      way   11.413552  120.426855  Topic45  -4.5337   1.4104

[1596 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
158       5  0.092139   able
158       6  0.046069   able
158       8  0.046069   able
158      11  0.138208   able
158      26  0.414625   able
...     ...       ...    ...
753      21  0.944687     yo
426      22  0.988386   york
507      26  0.960145  young
507      31  0.026671  young
212       8  0.951071   zone

[1973 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [26]:
# Save interactive pyLDAvis visualizations as html files
pyLDAvis.save_html(panel_k19, "../Data/pyldavis_k19.html")
pyLDAvis.save_html(panel_k45, "../Data/pyldavis_k45.html")

def summarize_topics(model: LdaModel, num_words: int = 12) -> pd.DataFrame:
    """Collect top-n keywords per topic for quick, non-interactive inspection."""
    rows = []
    for topic_id, word_info in model.show_topics(
        num_topics=-1, num_words=num_words, formatted=False
    ):
        # Join the per-topic (word, weight) tuples into a readable comma-separated string.
        tokens = ", ".join(word for word, _ in word_info)
        rows.append({"topic": topic_id, "keywords": tokens})
    return pd.DataFrame(rows).sort_values("topic")


print("Top keywords for k=19 model")
topics_k19 = summarize_topics(lda_runs[19], num_words=12)
display(topics_k19)

print("Top keywords for k=45 model")
topics_k45 = summarize_topics(lda_runs[45], num_words=12)
display(topics_k45)

Top keywords for k=19 model


,topic,keywords
0,0,"change, climate, check, brown, mask, country, ..."
1,1,"vote, fire, break, bad, movie, bring, set, mon..."
2,2,"team, win, game, good, go, time, league, year,..."
3,3,"red, heart, mark, family, love, double, sparkl..."
4,4,"love, man, look, get, watch, harry, fall, want..."
5,5,"hour, late, bay, wait, coach, daily, head, nfl..."
6,6,"week, social, half, tweet, lead, round, second..."
7,7,"day, happy, st, good, love, wish, enjoy, lockd..."
8,8,"news, open, say, trump, people, hear, god, rea..."
9,9,"star, power, war, month, ask, year, understand..."


Top keywords for k=45 model


,topic,keywords
0,0,"say, don, people, school, bill, government, le..."
1,1,"week, stream, find, home, away, way, try, flor..."
2,2,"go, look, good, boy, weekend, lose, to, forwar..."
3,3,"red, heart, sparkle, love, huge, good, true, r..."
4,4,"song, dance, twitter, app, tweet, party, hit, ..."
5,5,"thank, open, late, place, daily, didn, excited..."
6,6,"state, call, ohio, pick, playoff, protest, pay..."
7,7,"st, wish, coach, summer, service, learn, cowbo..."
8,8,"th, wait, head, hour, leave, follow, club, la,..."
9,9,"trump, power, president, easter, understand, b..."


---
## 5. Interpret Topics and Align Them with Dataset Labels

### 5.1 Topic → Label Alignment

Each tweet is assigned to the topic with the highest posterior probability and paired with its supervised label from the dataset. Aggregating these `(topic, label)` pairs lets us inspect, per topic, which labels dominate, how often a label appears, and how confident the model was when it picked that topic. The resulting table exposes:

- **Primary labels per topic** – high `count` + `topic_share` values show that, say, Topic 10 is mostly `sports`, while Topic 5 blends `diaries_&_daily_life` and `news_&_social_concern`.
- **Mixed topics** – topics whose top three labels all have similar shares indicate overlapping vocabulary or label definitions (useful for revisiting the taxonomy).
- **Confidence signals** – `avg_confidence` highlights whether the dominant label-topic pairing is strong (>0.4) or tentative (<0.25), guiding where further preprocessing or more topics might help.

In [27]:
def topic_label_alignment(model: LdaModel, corpus, labels, top_n: int = 3) -> pd.DataFrame:
    assignments = []
    for bow, label in zip(corpus, labels):
        topic_id, topic_prob = max(
            model.get_document_topics(bow, minimum_probability=0.0),
            key=lambda item: item[1],
        )
        assignments.append((topic_id, label, topic_prob))

    alignment = (
        pd.DataFrame(assignments, columns=["topic", "label", "probability"])
        .groupby(["topic", "label"])
        .agg(count=("probability", "size"),
             avg_confidence=("probability", "mean"))
        .reset_index()
    )
    alignment["topic_share"] = alignment.groupby("topic")["count"].transform(
        lambda counts: counts / counts.sum()
    )
    return alignment.sort_values(["topic", "count"], ascending=[True, False]) \
                    .groupby("topic").head(top_n)


print("Topic ↔ label alignment (k=19)")
alignment_k19 = topic_label_alignment(lda_runs[19], train_corpus, train_labels)
display(alignment_k19)

print("Topic ↔ label alignment (k=45)")
alignment_k45 = topic_label_alignment(lda_runs[45], train_corpus, train_labels)
display(alignment_k45)

Topic ↔ label alignment (k=19)


,topic,label,count,avg_confidence,topic_share
4,0,news_&_social_concern,171,0.468392,0.546326
0,0,celebrity_&_pop_culture,46,0.456204,0.146965
1,0,diaries_&_daily_life,35,0.403486,0.111821
10,1,news_&_social_concern,75,0.439523,0.270758
8,1,film_tv_&_video,72,0.486784,0.259928
7,1,diaries_&_daily_life,41,0.446835,0.148014
17,2,sports,337,0.475933,0.575085
16,2,news_&_social_concern,70,0.396207,0.119454
13,2,diaries_&_daily_life,57,0.460660,0.097270
19,3,diaries_&_daily_life,36,0.464097,0.290323


Topic ↔ label alignment (k=45)


,topic,label,count,avg_confidence,topic_share
4,0,news_&_social_concern,118,0.291747,0.548837
0,0,celebrity_&_pop_culture,27,0.305466,0.125581
1,0,diaries_&_daily_life,25,0.269342,0.116279
6,1,celebrity_&_pop_culture,24,0.295133,0.242424
7,1,diaries_&_daily_life,22,0.316984,0.222222
...,...,...,...,...,...
258,43,film_tv_&_video,51,0.360419,0.300000
259,43,music,49,0.406213,0.288235
266,44,news_&_social_concern,71,0.343144,0.533835
263,44,diaries_&_daily_life,25,0.319519,0.187970


### 5.2 Discussion and Observations

- **k = 19 (label-aligned)**: Most latent topics map cleanly to a single supervised class: `sports`, `music`, and `news_&_social_concern` dominate individual topics with ~70–80 % share. This makes the model easy to explain to stakeholders, yet some clusters still mix politics with general news or lifestyle chatter, showing that 19 topics can’t perfectly isolate every class.
- **k = 45 (metric best)**: Coherence improves and perplexity drops. The pyLDAvis view shows tightly packed bubbles for climate activism, NBA trade rumors, celebrity obituaries, pandemic updates, etc. These subtopics are great for exploratory analysis but fragment the supervised classes (multiple sports/politics bubbles), so this model trades simplicity for detail.
- **Topic divergences**: Several topics capture conversational filler (“good morning”, “happy birthday”) or sustained climate discussions that have no dedicated label, suggesting blind spots in the taxonomy. Climate-related tweets drifting away from general news at k=45 hint that an “environment” class could be useful.

### 5.3 How the Topics Inform the Dataset

- **Alignment with classes**: When k equals the class count, topics mostly reflect the label definitions, confirming that the preprocessing from earlier labs preserved the label structure.
- **Where to use which model**: Use the k=19 model when you need consistency with the supervised classifier or want to annotate new tweets with the existing classes. Use the k=45 model for editorial research, content audits, or to surface emerging subthemes ahead of creating new labels.
- **Practical takeaway**: Topic modeling highlights both well-covered areas (sports, entertainment) and underrepresented concepts (climate, casual chatter). This helps prioritize where to extend the label inventory or build specialized classifiers.

### 5.4 Visual Exploration

Inspect the exported pyLDAvis dashboards to drill into word distributions and bubble overlaps:

```bash
open grundlagen-des-nlp-ws25_26/Abgabe/Data/pyldavis_k19.html
open grundlagen-des-nlp-ws25_26/Abgabe/Data/pyldavis_k45.html
```

